In [ ]:
import os
import sys
import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sys.path.insert(0, '../src')
from classifier_results_metrics import output_metrics_for_augmentation, output_metrics_folds, output_metrics

In [ ]:
# def output_metrics_for_augmentation(path_name):
#     df_all = None
#     thresholds = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
#     for i in range(4):
#         file_names = glob.glob(f"{path_name}{i}/*.csv")
#         file_name = [x for x in file_names if f"fold{i}" in x][0]
#         # df = pr_auc_df(file_name, fold=i)
#         for t in thresholds:
#             df = output_metrics_folds(file_name, fold=i, label=f"threshold_{t}",
#                                       metrics=['mcc', 'pr_auc'], threshold=t)
#             if df_all is None:
#                 df_all = df
#             else:
#                 df_all = pd.concat([df_all, df])
#     return df_all.reset_index(drop=True)

dims = ["2D", "5D", "10D", "all"]
df_all = None
# base_path = "/home/sdan8/dataFusion/latent_space_performance_study/data/rotation_combined_aug_5i_circ_1/"
for dim in dims:
    # path_name = f"{base_path}outputs_rotation_C_{dim}_circ_1_fold"
    path_name = f"../outputs_rotation_C_{dim}/fold"
    df = output_metrics_for_augmentation(path_name)
    df["dim"] = dim
    # df.reset_index(drop=True)
    
    if df_all is None:
        df_all = df
    else:
        df_all = pd.concat([df_all, df])
df_all = df_all.reset_index(drop=True)
df_all

In [ ]:
baseline_path = "../outputs_rotation_0/fold"
df_baseline = output_metrics_for_augmentation(baseline_path).query("band == 'C'")
df_baseline["dim"] = "baseline"
df_baseline

In [ ]:
df_all = pd.concat([df_all, df_baseline]).reset_index(drop=True)

In [ ]:
hue_order = ["unet", "clss", "base"]
dims = ["2D", "5D", "10D", "all", "baseline"]
n = len(dims)
f, a = plt.subplots(n, 1, figsize=(13, n*4))
for i, dim in enumerate(dims):
    sns.boxplot(data=df_all.query(f"dim == '{dim}'"), ax=a[i], x="test_dir",
                y="mcc", hue="model", hue_order=hue_order)
    a[i].set_title(dim)
    a[i].axhline(y=0, c="k", ls='--',linewidth=0.5)
plt.show()

In [ ]:
# f, a = plt.subplots(1, 1, figsize=(13, 8))
# sns.boxplot(data=df_all.query("model == 'unet'"), ax=a, x="test_dir", y="mcc", hue="label")
# plt.show()

f, a = plt.subplots(n, 1, figsize=(13, n*4))
for i, dim in enumerate(dims):
    sns.boxplot(data=df_all.query(f"model == 'unet' and dim == '{dim}'"),
                ax=a[i], x="test_dir", y="mcc", hue="label")
    a[i].set_title(dim)
    a[i].axhline(y=0, c="k", ls='--',linewidth=0.5)
plt.show()

In [ ]:
rotations = [f"rotation_{x}" for x in [0, 90]]
n = len(rotations)
f, a = plt.subplots(n, 1, figsize=(21, n*5.5))
for i, rotation in enumerate(rotations):
    sns.barplot(data=df_all.query(f"model == 'unet' and test_dir == '{rotation}'"),
                ax=a[i], x="dim", y="mcc", hue="label")
    a[i].set_title(f"Rotation = {rotation.split('_')[1]}")
    a[i].axhline(y=0, c="k", ls='--',linewidth=0.5)
    a[i].set_ylabel("MCC")
    a[i].set_xlabel("")
fs = 16
plt.rc('xtick', labelsize=fs)
plt.rc('ytick', labelsize=fs)
plt.rc('axes', labelsize=fs)
plt.rc('figure', titlesize=20)
plt.rc('font', size=12)
plt.show()

In [ ]:
rotations = [f"rotation_{x}" for x in [0, 90]]
n = len(rotations)
f, a = plt.subplots(n, 1, figsize=(21, n*5))
for i, rotation in enumerate(rotations):
    sns.boxplot(data=df_all.query(f"model == 'unet' and test_dir == '{rotation}'"),
                ax=a[i], x="dim", y="mcc", hue="label")
    sns.stripplot(data=df_all.query(f"model == 'unet' and test_dir == '{rotation}'"),
                ax=a[i], x="dim", y="mcc", hue="label", color='k', dodge=True, jitter=0.3)
    a[i].set_title(rotation)
    a[i].axhline(y=0, c="k", ls='--',linewidth=0.5)
    handles, labels = a[i].get_legend_handles_labels()
    l = a[i].legend(handles[0:7], labels[0:7])
plt.show()

In [ ]:
import scipy.stats as st
def ci_95(data):
    ci = st.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=st.sem(data))
    interval = (ci[1] - ci[0]) / 2
    if interval is np.nan:
        return 0
    else:
        return interval

df_all.query("model == 'unet'").pivot_table(index=["dim", "label"],
                                            columns=["band", "test_dir"],
                                            values="mcc",
                                            aggfunc=["mean", ci_95])

In [ ]:
df_all["dim"].value_counts()